In [1]:
import os 
os.chdir('../')
%pwd

'/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareTimeseriesBaseModelConfig:
    root_dir: Path
    base_od_model_path: Path
    base_tensor_model_path: Path
    params_od_size: list
    params_tensor_size: list
    params_learning_rate: float
    params_time_lag : int

In [3]:
from timeseriesPredictor.constants import *
from timeseriesPredictor.utils import read_yaml, create_directories

In [4]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prepare_timeseries_base_model_config(self) -> PrepareTimeseriesBaseModelConfig:
        config = self.config.prepare_timeseries_base_model
        
        create_directories([config.root_dir])

        prepare_timeseries_base_model_config = PrepareTimeseriesBaseModelConfig(
            root_dir = config.root_dir,           
            base_od_model_path = config.base_od_model_path,    
            base_tensor_model_path = config.base_tensor_model_path,                   
            params_od_size = self.params.OD_SIZE,
            params_tensor_size= self.params.TENSOR_SIZE,
            params_learning_rate = self.params.LEARNING_RATE ,
            params_time_lag = self.params.TIME_LAG                

        )

        return prepare_timeseries_base_model_config

In [5]:
import keras
from timeseriesPredictor.logger import logging

2023-08-15 14:54:20.628342: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 14:54:20.812349: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 14:54:20.814161: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 14:54:22.170137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
class PrepareTimeseriesBaseModel:
    def __init__(self, config: PrepareTimeseriesBaseModelConfig):
        self.config = config

    @staticmethod
    def save_model(path:Path, model:keras.Model):
        model.save(path)

    @staticmethod
    def _prepare_full_model(learning_rate, input_shape):      
        
        # Construct the input layer with no definite frame size.
        inp = keras.layers.Input(shape= input_shape)

        # We will construct 3 `ConvLSTM2D` layers with batch normalization,
        # followed by a `Conv3D` layer for the spatiotemporal outputs.
        x = keras.layers.ConvLSTM2D(
            filters=64,
            kernel_size=(5, 5),
            padding="same",
            return_sequences=True,
            activation="relu",
        )(inp)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ConvLSTM2D(
            filters=64,
            kernel_size=(3, 3),
            padding="same",
            return_sequences=True,
            activation="relu",
        )(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ConvLSTM2D(
            filters=64,
            kernel_size=(1, 1),
            padding="same",
            return_sequences=True,
            activation="relu",
        )(x)
        x = keras.layers.Conv3D(
            filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
        )(x)

        # Next, we will build the complete model and compile it.
        model = keras.models.Model(inp, x)
        model.compile(
            loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
            )        
        
        model.summary()
        return model
    
    def update_model(self):
        input_shapes = [self.config.params_od_size, self.config.params_tensor_size]
        model_pathes = [self.config.base_od_model_path, self.config.base_tensor_model_path]
        
        for input_shape, model_path  in zip(input_shapes, model_pathes):
            input_shape = [self.config.params_time_lag -1] + input_shape            
            self.full_model = self._prepare_full_model(        
            learning_rate = self.config.params_learning_rate,
            input_shape = input_shape
            
            )

            self.save_model(path=model_path , model=self.full_model)
            logging.info(f"Timeseries base model saved at {model_path}!")

In [7]:
import sys
from timeseriesPredictor.exception import CustomException

In [8]:
try:
    config = configurationManeger()
    prepare_timeseries_base_model_config = config.get_prepare_timeseries_base_model_config()
    prepare_timeseries_base_model = PrepareTimeseriesBaseModel(config=prepare_timeseries_base_model_config)    
    prepare_timeseries_base_model.update_model()

except Exception as e:
    raise CustomException(e, sys)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 64, 64, 1)]    0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 6, 64, 64, 64)     416256    
                                                                 
 batch_normalization (Batch  (None, 6, 64, 64, 64)     256       
 Normalization)                                                  
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 6, 64, 64, 64)     295168    
                                                                 
 batch_normalization_1 (Bat  (None, 6, 64, 64, 64)     256       
 chNormalization)                                                
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 6, 64, 64, 64)     33024 

/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 6, 100, 64, 1)]   0         
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 6, 100, 64, 64)    416256    
                                                                 
 batch_normalization_2 (Bat  (None, 6, 100, 64, 64)    256       
 chNormalization)                                                
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, 6, 100, 64, 64)    295168    
                                                                 
 batch_normalization_3 (Bat  (None, 6, 100, 64, 64)    256       
 chNormalization)                                                
                                                                 
 conv_lstm2d_5 (ConvLSTM2D)  (None, 6, 100, 64, 64)    3302